In [ ]:
#updated
import numpy as np
from scipy.stats import mode
from sklearn.datasets import load_wine
from numpy.random import randint
from scipy.spatial.distance import cdist
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


def ova_svm_train(X, y, C, sigma):
    param_grid = {'C': C, 'gamma': 1 / (2 * sigma ** 2)}
    svm = SVC(kernel='rbf', **param_grid)
    svm.fit(X, y)
    return svm


def ova_svm_predict(model, X_test):
    return model.predict(X_test)


def active_learning(X_train, y_train, X_pool, y_pool, n_queries, C, sigma):
    # initialize the model
    model = ova_svm_train(X_train, y_train, C, sigma)
    n_classes = len(np.unique(y_train))
    # active learning loop
    for idx in range(n_queries):
        # compute the distance to the decision hyperplane for each instance in the pool
        dist = np.abs(model.decision_function(X_pool))

        # select the instance with the smallest distance
        if n_classes == 2:
            query_idx = np.argmin(np.abs(dist))
        else:
            query_idx = np.argmin(np.max(dist, axis=1))

        # update the model with the queried instance
        X_train = np.vstack([X_train, X_pool[query_idx]])
        y_train = np.hstack([y_train, y_pool[query_idx]])
        model = ova_svm_train(X_train, y_train, C, sigma)

        # remove the queried instance from the pool
        X_pool = np.delete(X_pool, query_idx, axis=0)
    
    return model


def accu_scr(y_test, y_pred):
    return np.mean(y_test == y_pred)


# Loading the Data
iris = load_wine()

# Store features matrix in X
X = iris.data
# Store target vector in
y = iris.target

r = int(input("Specify the size of training dataset: "))
m = int(input("Specify the size for active learning: "))
l = int(input("Specify the size of testing dataset: "))

C_range = np.logspace(-3, 3, 7)
sigma_range = np.logspace(-3, 3, 7)

counter = 0
for g in range(15):
    count = 0
    arr = []
    # Creating the training Data
    train_idx = []
    for p in range(3):
        if(p == 0):
            tt = xxx = randint(0 ,59 ,int(int(r)/3))
        if(p == 1):
            tt = xxx = randint(59 ,130 ,int(int(r)/3)) 
            
        if(p == 2):
            tt = xxx = randint(130 ,178 ,int(int(r)/3))    
        for w in range(len(tt)):
            train_idx.append(tt[w])
            
    X_train = X[train_idx]
    y_train = y[train_idx]

    # Creating the testing Data
    test_idx = randint(0, 178, l)
    X_test = X[test_idx]
    y_test = y[test_idx]

    # Perform grid search to find best hyperparameters
    svm = GridSearchCV(SVC(kernel='rbf'), {'C': C_range, 'gamma': 1 / (2 * sigma_range ** 2)}, cv=3)
    svm.fit(X_train, y_train)
    best_C = svm.best_params_['C']
    best_sigma = np.sqrt(1 / (2 * svm.best_params_['gamma']))

    # run active learning
    model = active_learning(X_train, y_train, X, y, m, best_C, best_sigma)

    # Applying the created function
    y_pred = ova_svm_predict(model, X_test)

    # Checking the accuracy
    a = accu_scr(y_test, y_pred)
    # keeping the value in an array
    arr.append(a)

    print("Accuracy for testcase", g + 1, "is:", a * 100, "%")

    # calculating mean accuracy
    counter = counter + a

overall_mean = counter / 15
print("Overall mean accuracy is:", overall_mean * 100)
